In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from map_util import *

In [ ]:
def generate_map(height:int=64, width:int=64, margin:int=1, max_rooms:int=30, max_size:int=15) -> tuple[np.ndarray, list, tuple, tuple]:
    # Map filled with walls
    map = np.ones((height, width), dtype=np.int8) * WALL

    def create_room(center_y, center_x, block_height, block_width):
        nonlocal map, height, width
        
        left = center_x - block_width // 2
        right = left + block_width
        top = center_y - block_height // 2
        bottom = top + block_height
        
        for y in range(top, bottom+1):
            for x in range(left, right+1):
                map[y, x] = EMPTY

    rooms = []

    while max_size > 4 and len(rooms) < max_rooms:
        bw = np.random.randint(3, max_size)
        bh = np.random.randint(3, max_size)

        map_choice = np.ones_like(map)

        # clear out edges
        for y in range(0, 1 + bh//2):
            for x in range(width):
                map_choice[y, x] = 0

        for y in range(height - 1 - (bh - bh//2), height):
            for x in range(width):
                map_choice[y, x] = 0

        for x in range(0, 1 + bw//2):
            for y in range(height):
                map_choice[y, x] = 0

        for x in range(width - 1 - (bw - bw//2), width):
            for y in range(height):
                map_choice[y, x] = 0

        # clear out other rooms
        for y in range(height):
            for x in range(0, width):
                if map[y,x] == EMPTY:
                    for cy in range(y - (bh-bh//2) - margin, y + bh//2 + margin + 1):
                        for cx in range(x - (bw-bw//2) - margin, x + bw//2 + margin + 1):
                            if 0 <= cy < height and 0 <= cx < width:
                                map_choice[cy, cx] = 0
                        
        I = np.argwhere(map_choice == 1)
        if len(I) == 0:
            max_size -= 1
            continue
        by,bx = I[np.random.choice(len(I))]

        create_room(by, bx, bh, bw)
        rooms.append((by, bx, bh, bw))

    rooms = np.array(rooms)

    todo = list(range(len(rooms)))

    # start with top left room
    current_index = np.argmin(np.array([math.sqrt(rooms[t][0]**2 + rooms[t][1]**2) for t in todo]))
    current = todo[current_index]

    while len(todo) > 1:
        current_y, current_x, _, _ = rooms[current]
        todo.remove(current)

        closest_indices = np.argsort([math.sqrt((rooms[t][0]-current_y)**2 + (rooms[t][1]-current_x)**2) for t in todo])[:2]
        next_index = np.random.choice(closest_indices)
        next = todo[next_index]

        # connect
        next_y, next_x, _, _ = rooms[next]

        if abs(next_x - current_x) > abs(next_y - current_y):
            if next_x != current_x:
                dir = np.sign(next_x - current_x)
                for x in range(current_x, next_x+dir, dir):
                    map[current_y, x] = EMPTY
                    map[current_y-dir, x] = EMPTY
            if next_y != current_y:
                dir = np.sign(next_y - current_y)
                for y in range(current_y, next_y+dir, dir):
                    map[y, next_x] = EMPTY
                    map[y, next_x-dir] = EMPTY
        else:
            if next_y != current_y:
                dir = np.sign(next_y - current_y)
                for y in range(current_y, next_y+dir, dir):
                    map[y, current_x] = EMPTY
                    map[y, current_x-dir] = EMPTY
            if next_x != current_x:
                dir = np.sign(next_x - current_x)
                for x in range(current_x, next_x+dir, dir):
                    map[next_y, x] = EMPTY
                    map[next_y-dir, x] = EMPTY

        current = next

    # place player in top left room
    top_left = np.argmin(np.array([math.sqrt(rx*rx + ry*ry) for ry,rx,_,_ in rooms]))
    player_room = rooms[top_left]
    ry, rx, rh, rw = player_room
    py = ry
    px = rx
    map[py, px] = PLAYER

    # place exit in bottom right room
    bottom_right = np.argmax(np.array([math.sqrt(rx*rx + ry*ry) for ry,rx,_,_ in rooms]))
    exit_room = rooms[bottom_right]
    ry, rx, rh, rw = exit_room
    ey = ry + (rh - rh//2)
    ex = rx + (rw - rw//2) + 1
    map[ey, ex] = EXIT
    
    return map, rooms, player_room, exit_room

In [ ]:
def get_pos(cell_type):
    global map
    return tuple(np.argwhere(map == cell_type)[0])

In [ ]:
def add_distances_from(distances:dict, position:tuple[int,int]):
    distances_extra,_ = compute_distances(map, position)

    distances_combined = {}
    for k in distances.keys():
        d = distances[k]
        if k in distances_extra:
            de = distances_extra[k]
            d += de
        distances_combined[k] = d
    return distances_combined

In [ ]:
def add_lock_around_exit(remaining_doors:list[int]):
    global map
    
    door_color = remaining_doors[np.random.choice(len(remaining_doors))]
    remaining_doors.remove(door_color)
    
    exit_y, exit_x = exit_pos = get_pos(EXIT)
    print(f'{exit_pos=}')
    
    if exit_y > 0 and map[exit_y-1, exit_x] == EMPTY:
        map[exit_y-1, exit_x] = door_color
    elif exit_y < map.shape[0]-1 and map[exit_y+1, exit_x] == EMPTY:
        map[exit_y+1, exit_x] = door_color
    elif exit_x > 0 and map[exit_y, exit_x-1] == EMPTY:
        map[exit_y, exit_x-1] = door_color
    elif exit_x < map.shape[1]-1 and map[exit_y, exit_x+1] == EMPTY:
        map[exit_y, exit_x+1] = door_color
        
    for y in range(exit_y-1, exit_y+2):
        for x in range(exit_x-1, exit_x+2):
            if 0 <= y < map.shape[0] and 0 <= x <map.shape[1] and map[y,x] == EMPTY:
                map[y,x] = WALL
        
    return door_color

In [ ]:
def get_room_distances(min_width:int, min_height:int) -> np.ndarray:
    global map, rooms
    distances = None
    positions = [tuple(pos) for pos in np.argwhere(np.logical_and(map != WALL, map != EMPTY))]
    for pos in positions:
        if distances is None:
            distances, _ = compute_distances(map, pos)
        else:
            distances = add_distances_from(distances, pos)
    for pos in positions:
        if pos in distances: del distances[pos]

    #plot_distances(distances, map.shape[0], map.shape[1])

    def get_room_distance(room):
        nonlocal distances, min_height, min_width

        ry, rx, rh, rw = room
        if rh < min_height or rw < min_width: return np.nan
        
        pos = (ry,rx)
        return distances[pos] if pos in distances else np.nan

    room_distances = np.array([get_room_distance(r) for r in rooms])
    return room_distances

def get_farthest_room(min_height:int, min_width:int) -> tuple[int,int,int,int]:
    global map, rooms
    room_distances = get_room_distances(min_height, min_width)
    room_index = np.nanargmax(room_distances)
    return rooms[room_index]

def get_random_pos_in_room(room, margin_y:int, margin_x:int) -> tuple[int,int]:
    ry,rx,rh,rw = room
    
    min_y = ry - rh//2 + margin_y
    max_y = ry + (rh - rh//2) - margin_y
    min_x = rx - rw//2 + margin_x
    max_x = rx + (rw - rw//2) - margin_x
    
    positions = [(y,x) for x in range(min_x, max_x+1) for y in range(min_y, max_y+1) if map[y,x] == EMPTY]
    return positions[np.random.choice(len(positions))]
    
def add_key_far(key_color):
    global map
    
    room = get_farthest_room(5, 5)
    pos = get_random_pos_in_room(room, 1, 1)

    map[pos[0], pos[1]] = key_color

In [ ]:
def save_map(map: np.ndarray[np.int8], path:str) -> None:
    h, w = map.shape
    data = bytearray()
    data.append(h & 0xFF)
    data.append(w & 0xFF)

    for y in range(h):
        for x in range(w):
            data.append(map[y,x])

    with open(path, 'wb') as file:
        file.write(data)

In [ ]:
door_to_key = {
    DOOR_RED: KEY_RED,
    DOOR_GREEN: KEY_GREEN,
    DOOR_BLUE: KEY_BLUE,
}
key_to_door = {v:k for k,v in door_to_key.items()}

In [ ]:
map, rooms, _, _ = generate_map(64, 64, margin=10, max_rooms=2)
plot_map(map)
save_map(map, '../level0.bin')

In [ ]:
map, rooms, _, _ = generate_map(64, 64, margin=2, max_rooms=12)
plot_map(map)
save_map(map, '../level1.bin')

In [ ]:
remaining_doors = [DOOR_RED, DOOR_GREEN, DOOR_BLUE]

map, rooms, _, _ = generate_map(64, 64, margin=2, max_rooms=40)
exit_door = add_lock_around_exit(remaining_doors)
exit_key = door_to_key[exit_door]
add_key_far(exit_key)
plot_map(map)
save_map(map, '../level2.bin')

In [ ]:
def add_key_locker(height, width, key_color, remaining_doors):
    global map
    
    room = get_farthest_room(height+2, width+2)
    center_y, center_x = get_random_pos_in_room(room, 1 + (height+1)//2, 1 + (width+1)//2)

    top = center_y - height//2
    bottom = top + height - 1
    left = center_x - width//2
    right = left + width - 1
    
    walls = []
    # side walls first
    for y in range(top + 1, bottom):
        walls.append((y, left))
        walls.append((y, right))
    for x in range(left + 1, right):
        walls.append((top, x))
        walls.append((bottom, x))

    # random door
    door = walls[np.random.choice(len(walls))]
    walls.remove(door)
    
    # add corners
    walls.append((top, left))
    walls.append((top, right))
    walls.append((bottom, right))
    walls.append((bottom, left))
    
    for y,x in walls:
        map[y,x] = WALL

    door_color = remaining_doors[np.random.choice(len(remaining_doors))]
    remaining_doors.remove(door_color)
    map[door[0],door[1]] = door_color

    map[center_y, center_x] = key_color 
    
    return door_color

In [ ]:
remaining_doors = [DOOR_RED, DOOR_GREEN, DOOR_BLUE]

map, rooms, _, _ = generate_map(64, 64, margin=2, max_rooms=40)

# add door around exit
exit_door = add_lock_around_exit(remaining_doors)
exit_key = door_to_key[exit_door]

# add locker for key
first_door = add_key_locker(5, 5, exit_key, remaining_doors)
first_key = door_to_key[first_door]

# add key
add_key_far(first_key)

plot_map(map)
save_map(map, '../level4.bin')

In [ ]:
def add_blacklock_around_exit():
    global map
    
    exit_y, exit_x = exit_pos = get_pos(EXIT)
    print(f'{exit_pos=}')
    
    if exit_y > 0 and map[exit_y-1, exit_x] == EMPTY:
        map[exit_y-1, exit_x] = DOOR_BLACK
    elif exit_y < map.shape[0]-1 and map[exit_y+1, exit_x] == EMPTY:
        map[exit_y+1, exit_x] = DOOR_BLACK
    elif exit_x > 0 and map[exit_y, exit_x-1] == EMPTY:
        map[exit_y, exit_x-1] = DOOR_BLACK
    elif exit_x < map.shape[1]-1 and map[exit_y, exit_x+1] == EMPTY:
        map[exit_y, exit_x+1] = DOOR_BLACK
        
    for y in range(exit_y-1, exit_y+2):
        for x in range(exit_x-1, exit_x+2):
            if 0 <= y < map.shape[0] and 0 <= x <map.shape[1] and map[y,x] == EMPTY:
                map[y,x] = WALL       

In [ ]:
remaining_doors = [DOOR_RED, DOOR_GREEN, DOOR_BLUE]

map, rooms, _, _ = generate_map(64, 64, margin=2, max_rooms=40)

# add door around exit
exit_door = add_blacklock_around_exit()

# add pressure plate next to door
door_pos = get_pos(DOOR_BLACK)
print(f'{door_pos=}')

if map[door_pos[0], door_pos[1]-2] == EMPTY:
    print('WEST')
    map[door_pos[0], door_pos[1]-2] = PRESSURE_PLATE
elif map[door_pos[0], door_pos[1]+2] == EMPTY:
    print('EAST')
    map[door_pos[0], door_pos[1]+2] = PRESSURE_PLATE
elif map[door_pos[0]-2, door_pos[1]] == EMPTY:
    print('NORTH')
    map[door_pos[0]-2, door_pos[1]] = PRESSURE_PLATE
elif map[door_pos[0]+2, door_pos[1]] == EMPTY:
    print('SOUTH')
    map[door_pos[0]+2, door_pos[1]] = PRESSURE_PLATE

plot_map(map)
save_map(map, '../level5.bin')

In [ ]:
def add_double_key_locker(key_color, remaining_doors):
    global map

    # Randomize orientation    
    orientation = 2#np.random.choice(4) # N, E, S, W
    height = 3 if orientation == 1 or orientation == 3 else 4
    width = 4 if orientation == 1 or orientation == 3 else 3
    
    door_inner = remaining_doors[np.random.choice(len(remaining_doors))]
    remaining_doors.remove(door_inner)
    door_outer = remaining_doors[np.random.choice(len(remaining_doors))]
    remaining_doors.remove(door_outer)
    
    room = get_farthest_room(height+2, width+2)
    center_y, center_x = get_random_pos_in_room(room, 1 + (height+1)//2, 1 + (width+1)//2)

    top = center_y - height//2
    bottom = top + height - 1
    left = center_x - width//2
    right = left + width - 1
    
    # Fill with walls
    for y in range(top, bottom + 1):
        for x in range(left, right + 1):
            map[y,x] = WALL
            
    # Final key in the center
   
    # Add double doors
    if orientation == 0: # North
        map[center_y, center_x] = key_color
        map[center_y-1, center_x] = door_inner
        map[center_y-2, center_x] = door_outer
    elif orientation == 1: # East
        map[center_y, center_x-1] = key_color
        map[center_y, center_x] = door_inner
        map[center_y, center_x+1] = door_outer
    elif orientation == 2: # South
        map[center_y-1, center_x] = key_color
        map[center_y, center_x] = door_inner
        map[center_y+1, center_x] = door_outer
    elif orientation == 3: # West
        map[center_y, center_x] = key_color
        map[center_y, center_x-1] = door_inner
        map[center_y, center_x-2] = door_outer

    return door_inner, door_outer, room

In [ ]:
def add_key_near(key_color, room):
    global map, rooms
    
    ry, rx, _, _ = room
    
    distances, _ = compute_distances(map, (ry, rx))
    
    closest_distance = None
    closest_room = None
    for other in rooms:
        if np.all(other == room): continue
        o = (other[0], other[1])
        d = distances[o] if o in distances else np.inf
        if closest_distance is None or d < closest_distance:
            closest_distance = d
            closest_room = other
            
    pos = get_random_pos_in_room(closest_room, 1, 1)
    map[pos[0], pos[1]] = key_color   


In [ ]:
def get_empty_in_front(pos):
    global map
    if map[pos[0]-1, pos[1]] == EMPTY:
        return (pos[0]-1, pos[1])
    if map[pos[0]+1, pos[1]] == EMPTY:
        return (pos[0]+1, pos[1])
    if map[pos[0], pos[1]-1] == EMPTY:
        return (pos[0], pos[1]-1)
    if map[pos[0], pos[1]+1] == EMPTY:
        return (pos[0], pos[1]+1)
    return None

In [ ]:
def find_room(pos):
    global rooms
    pos_y, pos_x = pos
    
    for room in rooms:
        ry,rx,rh,rw = room
        top = ry - rh // 2
        left = rx - rw // 2
        bottom = top + rh
        right = left + rw
        
        if top <= pos_y < bottom and left <= pos_x < right:
            return room
        
    return None      

In [ ]:
def remove_room(rooms, room):
    return rooms[np.any(rooms != room, axis=-1)]

In [ ]:
def get_closest_room(available_rooms, from_pos):
    global map
    distances, _ = compute_distances(map, from_pos)
    min_dist = None
    min_room = None
    for room in available_rooms:
        ry,rx,_,_ = room
        p = (ry,rx)
        if p not in distances: continue
        d = distances[p]
        if min_dist is None or d < min_dist:
            min_dist = d
            min_room = room
    return min_room

In [ ]:
map, rooms, player_room, exit_room = generate_map(48, 48, margin=2, max_rooms=40, max_size=10)

remaining_doors = [DOOR_RED, DOOR_GREEN, DOOR_BLUE]
available_rooms = rooms.copy()
available_rooms = remove_room(available_rooms, player_room)
available_rooms = remove_room(available_rooms, exit_room)

# add door around exit
exit_door = add_lock_around_exit(remaining_doors)
exit_key = door_to_key[exit_door]

# add locker for key
inner_door, outer_door, locker_room = add_double_key_locker(exit_key, remaining_doors)
inner_key = door_to_key[inner_door]
outer_key = door_to_key[outer_door]

exit_key_pos = get_pos(exit_key)
locker_room = find_room(exit_key_pos)
available_rooms = remove_room(available_rooms, locker_room)

# add keys

# find pos in front of outer door
player_pos = get_pos(PLAYER)
distances_p, _ = compute_distances(map, player_pos)
outer_pos = get_pos(outer_door)
outer_front_pos = get_empty_in_front(outer_pos)
distances_d, _ = compute_distances(map, outer_front_pos)

min_diff = None
center_positions = []
for pos,dist_p in distances_p.items():
    if pos not in distances_d: continue

    dist_d = distances_d[pos]
    
    diff = abs(dist_d - dist_p)
    if min_diff is None or diff < min_diff:
        min_diff = diff
        center_positions = [pos]
    elif min_diff is not None and diff == min_diff:
        center_positions.append(pos)
 
center_pos = center_positions[np.argmin([distances_p[p] for p in center_positions])]
map[center_pos[0], center_pos[1]] = inner_key

inner_key_room = find_room(center_pos)
available_rooms = remove_room(available_rooms, inner_key_room)

exit_door_pos = get_pos(exit_door)
infront_exit_door = get_empty_in_front(get_pos(exit_door))
outer_key_room = get_closest_room(available_rooms, infront_exit_door)
outer_key_pos = get_random_pos_in_room(outer_key_room, 1, 1)
map[outer_key_pos[0], outer_key_pos[1]] = outer_key


plot_map(map)
save_map(map, '../level6.bin')